In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from startup import np, pd, plt, sns
from startup_gui import widgets, interact, interactive

In [3]:
import pzflow
import tensorflow_probability.substrates.jax as tfp

In [140]:
import jax

In [141]:
import jax.numpy as jnp

In [4]:
import os
from io import StringIO
from dataclasses import dataclass

In [16]:
from sklearn.model_selection import train_test_split

In [5]:
from journal_20231211_nf_functions import MafData, MafDataOriginal

In [132]:
tfd = tfp.distributions
tfb = tfp.bijectors

In [ ]:
# TensorFlow Probability 

batch_size = 512
key, subkey = jax.random.split(jax.random.PRNGKey(42))
x2_dist = tfd.Normal(loc=0., scale=4.)
x2_samples = x2_dist.sample(batch_size, seed=subkey)
x1_dist = tfd.Normal(loc=0.25 * jnp.square(x2_samples), scale=jnp.ones(batch_size, dtype=jnp.float32))
key, subkey = jax.random.split(key)
x1_samples = x1_dist.sample(seed=key)
x_sample = jnp.stack([x1_samples, x2_samples], axis=1)
plt.scatter(x_sample[:, 0], x_sample[:, 1])
dist = tfd.JointDistributionSequential(
    [
        tfd.Normal(loc=0., scale=4., name='x2'),
        lambda x2: tfd.Normal(loc=0.25 * jnp.square(x2), scale=1., name='x1')
    ],
    use_vectorized_map=True, batch_ndims=0
)
print(f'Batch shape: {dist.batch_shape}, Event shape: {dist.event_shape}')
x2, x1 = dist.sample(batch_size, seed=key)
plt.scatter(x1, x2)
dist.resolve_graph()
x2.shape
# indep_dist = tfd.Independent(distribution=dist, reinterpreted_batch_ndims=1)
# print(f'Batch shape: {indep_dist.batch_shape}, Event shape: {indep_dist.event_shape}')